In [8]:
!find / -name "libturbojpeg.so*" 2>/dev/null


/workspace/miniconda3/pkgs/libjpeg-turbo-3.1.2-hb03c661_0/lib/libturbojpeg.so.0
/workspace/miniconda3/pkgs/libjpeg-turbo-3.1.2-hb03c661_0/lib/libturbojpeg.so.0.4.0
/workspace/miniconda3/pkgs/libjpeg-turbo-3.1.2-hb03c661_0/lib/libturbojpeg.so


In [4]:
! pip install pyTurboJPEG


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyTurboJPEG: filename=pyturbojpeg-1.8.2-py3-none-any.whl size=13189 sha256=16f1e07f824a6483a8e4b9adc535d3b6eeb4830d4cf1612513739f223c11b90b
  Stored in directory: /home/user4/.cache/pip/wheels/d7/63/c1/2a744d6efe09cabeda037cd067956c9948371347704edc6fee
Successfully built pyTurboJPEG


In [ ]:
# ============================================================
# 0. 기본 환경 + SEED 고정
# ============================================================
import sys, os
import random
import numpy as np
import torch
from tqdm import tqdm

# SEED
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# cuDNN reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# ============================================================
# 1. Dataset (Pillow)
# ============================================================
from PIL import Image
from torch.utils.data import Dataset

class CleanHappyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = []
        self.labels = []
        self.transform = transform

        for emo in os.listdir(root_dir):
            emo_dir = os.path.join(root_dir, emo)
            if not os.path.isdir(emo_dir):
                continue

            label = 1 if emo == "happy" else 0

            for f in os.listdir(emo_dir):
                if f.lower().endswith(("jpg", "jpeg", "png")):
                    p = os.path.join(emo_dir, f)
                    try:
                        Image.open(p)
                        self.paths.append(p)
                        self.labels.append(label)
                    except:
                        continue

        print(f"Loaded {len(self.paths)} images from {root_dir}")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        label = self.labels[idx]

        try:
            img = np.array(Image.open(p).convert("RGB"))
        except:
            return self.__getitem__((idx + 1) % len(self.paths))

        if self.transform:
            img = self.transform(image=img)["image"]

        return img, torch.tensor(label).long()


# ============================================================
# 2. Transform (Normalize X)
# ============================================================
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std =[0.229, 0.224, 0.225]),
    ToTensorV2(),
])


# ============================================================
# 3. Dataloader
# ============================================================
train_root = "/workspace/merge_data_binary/new_data/img_binary"
val_root   = "/workspace/merge_data_binary/new_data_val/img_binary"

train_dataset = CleanHappyDataset(train_root, transform)
val_dataset   = CleanHappyDataset(val_root, transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True,
    num_workers=8, pin_memory=True, persistent_workers=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=32, shuffle=False,
    num_workers=8, pin_memory=True, persistent_workers=True
)


# ============================================================
# 4. EfficientFormer (L1 → 448차원)
# ============================================================
sys.path.append("/workspace/EfficientFormer")
from models.efficientformer import efficientformer_l1

model = efficientformer_l1(pretrained=True)
model.classifier = torch.nn.Linear(448, 2)
model = model.to(device)


# ============================================================
# 5. Optimizer & Loss
# ============================================================
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5, weight_decay=1e-5)
scaler = torch.amp.GradScaler("cuda")


# ============================================================
# 6. Metrics 지원 함수
# ============================================================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(labels, preds):
    acc  = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec  = recall_score(labels, preds, zero_division=0)
    f1   = f1_score(labels, preds, zero_division=0)
    return acc, prec, rec, f1


# ============================================================
# 7. Train / Validate
# ============================================================
def train_one_epoch():
    model.train()
    total_loss = 0
    preds_all, labels_all = [], []

    for imgs, labels in tqdm(train_loader, desc="Train", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            outputs = model(imgs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        preds = outputs.argmax(1)

        total_loss += loss.item() * labels.size(0)
        preds_all.extend(preds.cpu().numpy())
        labels_all.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(train_dataset)
    acc, prec, rec, f1 = compute_metrics(labels_all, preds_all)

    return avg_loss, acc, prec, rec, f1


def validate():
    model.eval()
    total_loss = 0
    preds_all, labels_all = [], []

    with torch.inference_mode(), torch.amp.autocast("cuda"):
        for imgs, labels in tqdm(val_loader, desc="Validation", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]

            loss = criterion(outputs, labels)
            preds = outputs.argmax(1)

            total_loss += loss.item() * labels.size(0)
            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(val_dataset)
    acc, prec, rec, f1 = compute_metrics(labels_all, preds_all)

    return avg_loss, acc, prec, rec, f1


# ============================================================
# 8. Train Loop with Metrics
# ============================================================
num_epochs = 10
best_acc = 0

for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")

    train_loss, train_acc, train_prec, train_rec, train_f1 = train_one_epoch()
    val_loss, val_acc, val_prec, val_rec, val_f1 = validate()

    print(f"[Train] Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_prec:.4f} | Rec: {train_rec:.4f} | F1: {train_f1:.4f}")
    print(f"[Val]   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_rec:.4f} | F1: {val_f1:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "efficientformer_clean_best.pth")
        print("💾 Best model saved!")

print(f"\n🎉 Training Completed — Best Val Acc: {best_acc:.4f}")


Using device: cuda
Loaded 9550 images from /workspace/merge_data_binary/new_data/img_binary
Loaded 1151 images from /workspace/merge_data_binary/new_data_val/img_binary

===== Epoch 1/10 =====


NameError: name 'tqdm' is not defined

In [1]:
! pip install pytorch-lightning transformers timm albumentations accelerate
! pip install lightning torchvision torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 178.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 293.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 205.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [pytorch-lightning]pytorch-lightning]s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 14.7 MB/s  0:00:00


In [ ]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from tqdm import tqdm
from transformers import (
    EfficientFormerImageProcessor,
    EfficientFormerForImageClassification,
)

# =====================================================
# 1. SEED 고정
# =====================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"


# =====================================================
# 2. DATASET
# =====================================================
class HappyDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.root_dir = root_dir
        self.processor = processor
        self.samples = []

        classes = sorted(os.listdir(root_dir))  # ["happy", "other"]
        self.class_to_idx = {c: i for i, c in enumerate(classes)}

        for cls in classes:
            folder = os.path.join(root_dir, cls)
            for file in os.listdir(folder):
                if file.lower().endswith(("jpg", "jpeg", "png")):
                    self.samples.append(
                        (os.path.join(folder, file), self.class_to_idx[cls])
                    )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        processed = self.processor(images=img, return_tensors="pt")
        pixel_values = processed["pixel_values"].squeeze(0)

        return pixel_values, torch.tensor(label)


# =====================================================
# 3. LOAD Processor & Model
# =====================================================
processor = EfficientFormerImageProcessor.from_pretrained(
    "snap-research/efficientformer-l1-300"
)

model = EfficientFormerForImageClassification.from_pretrained(
    "snap-research/efficientformer-l1-300",
    num_labels=2,
    ignore_mismatched_sizes=True
)

model.to(device)


# =====================================================
# 4. PATHS
# =====================================================
train_root = "/workspace/merge_data_binary/new_data/img_binary"
val_root   = "/workspace/merge_data_binary/new_data_val/img_binary"

train_dataset = HappyDataset(train_root, processor)
val_dataset   = HappyDataset(val_root, processor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)


# =====================================================
# 5. Optimizer
# =====================================================
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)


# =====================================================
# 6. TRAINING LOOP
# =====================================================
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training")

    for pixel_values, labels in pbar:
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

    # ⭐ FIXED: 평균 loss 계산 ⭐
    avg_loss = total_loss / len(train_loader)
    print(f"\n[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}")

    # -------------------------
    # Validation
    # -------------------------
    model.eval()
    preds, gts = [], []

    with torch.no_grad():
        for pixel_values, labels in tqdm(val_loader, desc="Validating"):
            pixel_values = pixel_values.to(device)

            outputs = model(pixel_values=pixel_values)
            pred = outputs.logits.argmax(dim=1).cpu()

            preds.extend(pred.tolist())
            gts.extend(labels.tolist())

    # ============================
    # METRICS
    # ============================
    acc = accuracy_score(gts, preds)
    prec = precision_score(gts, preds, average="binary")
    rec = recall_score(gts, preds, average="binary")
    f1 = f1_score(gts, preds, average="binary")
    cm = confusion_matrix(gts, preds)

    print("\n========== METRICS ==========")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {prec:.4f}")
    print(f"Recall    : {rec:.4f}")
    print(f"F1-Score  : {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("============================\n")


Some weights of EfficientFormerForImageClassification were not initialized from the model checkpoint at snap-research/efficientformer-l1-300 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 448]) in the checkpoint and torch.Size([2, 448]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[Epoch 1] Training: 100%|██████████| 299/299 [05:57<00:00,  1.20s/it, loss=0.396] 



[Epoch 1] Train Loss: 137.9805


Validating: 100%|██████████| 36/36 [00:45<00:00,  1.25s/it]



========== METRICS ==========
Accuracy  : 0.9062
Precision : 0.9769
Recall    : 0.8944
F1-Score  : 0.9338
Confusion Matrix:
[[281  18]
 [ 90 762]]



[Epoch 2] Training: 100%|██████████| 299/299 [05:58<00:00,  1.20s/it, loss=0.0108]


...


========== METRICS ==========
Accuracy  : 0.9340
Precision : 0.9826
Recall    : 0.9272
F1-Score  : 0.9541
Confusion Matrix:
[[285  14]
 [ 62 790]]



[Epoch 5] Training: 100%|██████████| 299/299 [05:55<00:00,  1.19s/it, loss=0.00109] 



[Epoch 5] Train Loss: 8.3629


Validating: 100%|██████████| 36/36 [00:45<00:00,  1.26s/it]


========== METRICS ==========
Accuracy  : 0.9374
Precision : 0.9839
Recall    : 0.9308
F1-Score  : 0.9566
Confusion Matrix:
[[286  13]
 [ 59 793]]

